# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [8]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [9]:
city_data = pd.read_csv("..\WeatherPy\Output\City_Data_Output.csv")
city_data.head()

,Unnamed: 0,City,Lat,Lng,Max Temp,Min Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,hobart,-42.88,147.33,64.99,60.80,72,75,1.12,AU,1609711992
1,1,qaanaaq,77.48,-69.36,2.01,2.01,83,82,3.53,GL,1609712078
2,2,lavrentiya,65.58,-171.00,-6.02,-6.02,90,85,15.14,RU,1609712079
3,3,tiksi,71.69,128.87,-23.73,-23.73,84,43,3.38,RU,1609712080
4,4,wattegama,6.80,81.48,75.20,75.20,100,75,5.82,LK,1609712082


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [11]:
gmaps.configure(api_key=g_key)
locations = city_data[["Lat","Lng"]]
humidity = city_data["Humidity"].astype(float)

fig = gmaps.figure()

#heat_layer = gmaps.heatmap_layer(locations, weights=humidity,dissipating=False,max_intensity=100,point_radius=1)

#fig.add_layer(heat_layer)
fig


Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
